In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv
/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00005.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00057.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00020.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00072.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00040.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00043.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00010.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original/disco/disco00071.png
/kaggle/input/gtzan-dataset-music-genre-classification/Data/image

In [3]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 70.3 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.5 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.11-py3-none-any.whl size=5595886 sha256=c423f8bcb9f527931f60e3a70b94af15905bd7d5cc70c9080236caa3a6c53de1
  Stored in directory: /root/.cache/pip/wheels/09/e6/e6/29223dbea25e71e517b8791bf35cc9a7b872cb2ad284e30181
Successfully built pretty_midi


In [5]:
!pip install basic_pitch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.3/758.3 kB 18.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Su

In [10]:
import concurrent.futures
import os
import glob
import pandas as pd
import pretty_midi
from basic_pitch.inference import predict
from tqdm import tqdm

BASE_PATH = '/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original'
OUTPUT_PATH = '/kaggle/working/genre_excels'
os.makedirs(OUTPUT_PATH, exist_ok=True)
MAX_WORKERS = 10

# Gather files by genre
genre_audio_map = {}
for genre in sorted(os.listdir(BASE_PATH)):
    genre_dir = os.path.join(BASE_PATH, genre)
    if os.path.isdir(genre_dir):
        files = glob.glob(os.path.join(genre_dir, '*.wav'))
        genre_audio_map[genre] = files


In [11]:
def transcribe_single_file(audio_path, genre):
    try:
        output = predict(audio_path)
        note_events = output[2]
        df = pd.DataFrame(note_events, columns=['start', 'end', 'pitch_midi', 'velocity', 'mask'])
        df['track'] = os.path.basename(audio_path)
        df['genre'] = genre
        df['note_name'] = df['pitch_midi'].apply(pretty_midi.note_number_to_name)
        return df
    except Exception as e:
        print(f"Failed on {audio_path}: {e}")
        return None


In [12]:
for genre, audio_files in genre_audio_map.items():
    print(f"Processing genre: {genre}, {len(audio_files)} files")
    dfs = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(tqdm(executor.map(lambda f: transcribe_single_file(f, genre), audio_files),
                            total=len(audio_files), desc=f"Genre {genre}"))
        for df in results:
            if df is not None:
                dfs.append(df)
    if dfs:
        genre_df = pd.concat(dfs, ignore_index=True)
        genre_excel_path = os.path.join(OUTPUT_PATH, f"{genre}.xlsx")
        genre_df.to_excel(genre_excel_path, index=False)
        print(f"Saved genre {genre} to {genre_excel_path}")
    else:
        print(f"No transcriptions for genre {genre}")


Processing genre: blues, 100 files
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00039.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00027.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00005.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00058.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00028.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00024.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00062.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00090.wav.

Genre blues:   1%|          | 1/100 [00:49<1:21:54, 49.64s/it]

Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00002.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00095.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00063.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00094.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00059.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00006.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00079.wav...


Genre blues:   3%|▎         | 3/100 [01:01<24:01, 14.86s/it]  

Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00069.wav...


...

Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/rock/rock.00050.wav...
Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/rock/rock.00024.wav...


Genre rock:  90%|█████████ | 90/100 [09:28<00:36,  3.65s/it]

Predicting MIDI for /kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/rock/rock.00042.wav...


Genre rock: 100%|██████████| 100/100 [10:06<00:00,  6.07s/it]


Saved genre rock to /kaggle/working/genre_excels/rock.xlsx
